In [1]:
!pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.3 MB/s eta 0:00:00


In [2]:
# Imports and optional installs
from mtcnn import MTCNN
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import random
import warnings
warnings.filterwarnings('ignore')

2025-11-30 10:50:54.221072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764499854.627770      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764499854.761292      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# Configurable parameters (edit these before running)
# When running on Kaggle, save outputs under /kaggle/working/output for persistence
IS_KAGGLE = os.path.exists('/kaggle/input')

if IS_KAGGLE:
    ROOT_DATA_DIR = '/kaggle/working/output'
    print('Running in Kaggle — outputs will be saved to', ROOT_DATA_DIR)
else:
    ROOT_DATA_DIR = os.path.abspath('data')  # base for images and metadata

IMAGES_DIR = os.path.join(ROOT_DATA_DIR, 'images')
METADATA_DIR = os.path.join(ROOT_DATA_DIR, 'metadata')
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(METADATA_DIR, exist_ok=True)

# Source dataset root (update for your environment)
CELEBDF_PATH = '/kaggle/input/celeb-df-v2'

# Extraction options
DEFAULTS = {
    'max_videos_per_type': None,       # None = all
    'frames_per_video': 10,
    'img_size': 256,
    'face_margin': 20,
    'min_confidence': 0.0,
    'resize_max_width': 640,
    'save_format': 'png',
    'overwrite': False,  # if True, re-extract and overwrite existing images
    'max_fake_videos_per_real': 2  # max number of fake videos to extract per real video for balanced data
}

print('Default extraction options: ', DEFAULTS)
print('Images will be saved to:', IMAGES_DIR)
print('Metadata will be saved to:', METADATA_DIR)


Running in Kaggle — outputs will be saved to /kaggle/working/output
Default extraction options:  {'max_videos_per_type': None, 'frames_per_video': 10, 'img_size': 256, 'face_margin': 20, 'min_confidence': 0.0, 'resize_max_width': 640, 'save_format': 'png', 'overwrite': False, 'max_fake_videos_per_real': 2}
Images will be saved to: /kaggle/working/output/images
Metadata will be saved to: /kaggle/working/output/metadata


In [4]:
def _ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def _save_image(image_rgb, out_path, fmt='png', jpeg_quality=95):
    # image_rgb: HxWx3 in RGB. cv2.imwrite expects BGR.
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    if fmt.lower() in ['jpg', 'jpeg']:
        cv2.imwrite(out_path, image_bgr, [cv2.IMWRITE_JPEG_QUALITY, jpeg_quality])
    else:
        cv2.imwrite(out_path, image_bgr)

def extract_frames_from_video_to_files(video_path, out_dir, video_id=None,
                                    num_frames=10, face_detector=None, img_size=256,
                                    face_margin=20, min_confidence=0.9, resize_max_width=640,
                                    save_format='png', overwrite=False, jpeg_quality=95):
    """
    Extracts evenly-spaced frames from `video_path`, detects the largest face per frame,
    writes face crops to `out_dir`, and returns a list of metadata dicts.
    
    Uses MTCNN for face detection.

    Returns list of dicts: {filepath,video_id,frame_idx,bbox_x,bbox_y,bbox_w,bbox_h,confidence,timestamp}
    """
    
    # Initialize MTCNN detector if not provided
    if face_detector is None:
        face_detector = MTCNN()
    
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    fps = cap.get(cv2.CAP_PROP_FPS) if cap.get(cv2.CAP_PROP_FPS) else None

    if total_frames == 0:
        cap.release()
        return []

    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    rows = []

    for i, frame_idx in enumerate(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_idx))
        ret, frame = cap.read()
        if not ret:
            continue

        h, w = frame.shape[:2]
        if w > resize_max_width:
            scale = resize_max_width / float(w)
            frame = cv2.resize(frame, (resize_max_width, int(h * scale)), interpolation=cv2.INTER_AREA)
            h, w = frame.shape[:2]

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        try:
            # MTCNN returns a list of face detections
            # Each detection has: 'box' (x, y, width, height), 'confidence', 'keypoints'
            detections = face_detector.detect_faces(frame_rgb)
        except Exception:
            continue

        if not detections or len(detections) == 0:
            continue

        # Select the largest face by area
        best_face = None
        best_area = -1
        
        for detection in detections:
            box = detection.get('box', None)
            confidence = detection.get('confidence', 0.0)
            
            if box is None or confidence < min_confidence:
                continue
            
            # box is [x, y, width, height]
            x, y, width, height = box
            area = width * height
            
            if area > best_area:
                best_area = area
                best_face = (float(x), float(y), float(x + width), float(y + height), float(confidence))

        if best_face is None:
            continue

        x1, y1, x2, y2, conf = best_face

        # Convert to ints and apply margin
        x1i = int(round(x1))
        y1i = int(round(y1))
        x2i = int(round(x2))
        y2i = int(round(y2))

        x0 = max(0, x1i - face_margin)
        y0 = max(0, y1i - face_margin)
        x1c = min(frame_rgb.shape[1], x2i + face_margin)
        y1c = min(frame_rgb.shape[0], y2i + face_margin)

        face = frame_rgb[y0:y1c, x0:x1c]
        if face.size == 0:
            continue

        face_resized = cv2.resize(face, (img_size, img_size), interpolation=cv2.INTER_AREA)

        # prepare filename
        vid = video_id if video_id is not None else os.path.splitext(os.path.basename(video_path))[0]
        filename = f"{vid}_{i:04d}.{save_format}"
        out_path = os.path.join(out_dir, filename)

        if not overwrite and os.path.exists(out_path):
            rows.append({
                'filepath': os.path.relpath(out_path),
                'video_id': vid,
                'frame_idx': int(frame_idx),
                'bbox_x': int(x0),
                'bbox_y': int(y0),
                'bbox_w': int(x1c - x0),
                'bbox_h': int(y1c - y0),
                'confidence': conf,
                'timestamp': float(frame_idx) / fps if fps else None,
            })
        else:
            _ensure_dir(os.path.dirname(out_path))
            _save_image(face_resized, out_path, fmt=save_format, jpeg_quality=jpeg_quality)
            rows.append({
                'filepath': os.path.relpath(out_path),
                'video_id': vid,
                'frame_idx': int(frame_idx),
                'bbox_x': int(x0),
                'bbox_y': int(y0),
                'bbox_w': int(x1c - x0),
                'bbox_h': int(y1c - y0),
                'confidence': conf,
                'timestamp': float(frame_idx) / fps if fps else None,
            })

    cap.release()
    return rows

In [5]:
def process_celebdf_to_images(celebdf_root, out_base_dir, metadata_dir,
                             max_videos_per_subject=None, frames_per_video=10, img_size=256,
                             face_margin=20, min_confidence=0.9,
                             resize_max_width=640, save_format='png', overwrite=False,
                             max_fake_videos_per_real=1, jpeg_quality=95):
    """
    Process Celeb-DF dataset: extracts faces from real videos in `Celeb-real`
    and matched synthesis videos in `Celeb-synthesis`.

    Matching rule: for a real filename like `id0_0000.mp4`, subject prefix is `id0`.
    Fake videos matching `id0_*` will be randomly sampled (up to max_fake_videos_per_real)
    to create a balanced dataset.
    
    Uses MTCNN for face detection.

    Args:
        max_fake_videos_per_real: Maximum number of fake videos to extract per real video.
                                  Set to 1 for 1:1 balance, 2 for 2:1 ratio, None for all fake videos.

    Returns a pandas.DataFrame and saves `labels_celebdf.csv` + `detections_celebdf.json` into `metadata_dir`.
    """
    
    # Initialize MTCNN detector once for all videos
    print('Initializing MTCNN detector...')
    face_detector = MTCNN()
    print('Face detector initialized.')
    
    rows = []

    celeb_real_dir = os.path.join(celebdf_root, 'Celeb-real')
    celeb_synth_dir = os.path.join(celebdf_root, 'Celeb-synthesis')

    if not os.path.exists(celeb_real_dir):
        print('Celeb-DF real directory not found at', celeb_real_dir)
        return pd.DataFrame()

    real_videos = [f for f in os.listdir(celeb_real_dir) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    if max_videos_per_subject is not None:
        real_videos = real_videos[:max_videos_per_subject]
    print(f'Processing {len(real_videos)} Celeb-DF real videos from {celeb_real_dir}')

    out_celeb_real = os.path.join(out_base_dir, 'celebdf', 'real')
    _ensure_dir(out_celeb_real)

    for vf in tqdm(real_videos, desc='celeb_real'):
        video_path = os.path.join(celeb_real_dir, vf)
        try:
            new_rows = extract_frames_from_video_to_files(video_path, out_celeb_real, video_id=os.path.splitext(vf)[0],
                                                        num_frames=frames_per_video, face_detector=face_detector,
                                                        img_size=img_size, face_margin=face_margin,
                                                        min_confidence=min_confidence, resize_max_width=resize_max_width,
                                                        save_format=save_format, overwrite=overwrite, jpeg_quality=jpeg_quality)
        except Exception as e:
            print('Error processing', video_path, e)
            continue
        for r in new_rows:
            r['label'] = 0
            r['method'] = 'celeb_real'
        rows.extend(new_rows)

    # Find matching synth files
    if os.path.exists(celeb_synth_dir):
        synth_files = [f for f in os.listdir(celeb_synth_dir) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        out_celeb_synth = os.path.join(out_base_dir, 'celebdf', 'synthesis')
        _ensure_dir(out_celeb_synth)
        
        for vf in tqdm(real_videos, desc='celeb_map_synth'):
            subject = os.path.splitext(vf)[0].split('_')[0]
            matched = [s for s in synth_files if s.startswith(subject + '_')]
            
            # For balanced data: randomly sample fake videos if max_fake_videos_per_real is set
            if max_fake_videos_per_real is not None and len(matched) > max_fake_videos_per_real:
                matched = random.sample(matched, max_fake_videos_per_real)
            
            for sf in matched:
                video_path = os.path.join(celeb_synth_dir, sf)
                try:
                    new_rows = extract_frames_from_video_to_files(video_path, out_celeb_synth, video_id=os.path.splitext(sf)[0],
                                                                num_frames=frames_per_video, face_detector=face_detector,
                                                                img_size=img_size, face_margin=face_margin,
                                                                min_confidence=min_confidence, resize_max_width=resize_max_width,
                                                                save_format=save_format, overwrite=overwrite, jpeg_quality=jpeg_quality)
                except Exception as e:
                    print('Error processing', video_path, e)
                    continue
                for r in new_rows:
                    r['label'] = 1
                    r['method'] = 'celeb_synthesis'
                rows.extend(new_rows)
    else:
        print('Celeb-DF synthesis directory not found at', celeb_synth_dir)

    if not rows:
        print('No face crops extracted for Celeb-DF.')
        return pd.DataFrame()

    df = pd.DataFrame(rows)
    df['filepath'] = df['filepath'].apply(lambda p: os.path.normpath(p))

    # Save Celeb-DF specific metadata
    labels_csv = os.path.join(metadata_dir, 'labels_celebdf.csv')
    df.to_csv(labels_csv, index=False)
    detections_json = os.path.join(metadata_dir, 'detections_celebdf.json')
    with open(detections_json, 'w') as f:
        json.dump(rows, f, indent=2)

    print('Saved', len(df), 'rows to', labels_csv)
    return df

In [6]:
# Example run cell: edit parameters and run this cell to extract faces to disk
opts = dict(DEFAULTS)
opts['max_videos_per_type'] = None  # None = process all 590 real videos
opts['frames_per_video'] = 10
opts['img_size'] = 256  # Good balance for both ResNet (224) and EfficientNet-B4 (380)
opts['face_margin'] = 15
opts['min_confidence'] = 0.8  # MTCNN confidence threshold (0.8 is good)
opts['resize_max_width'] = 640
opts['save_format'] = 'jpg'  # Faster I/O
opts['jpeg_quality'] = 90
opts['overwrite'] = False
opts['max_fake_videos_per_real'] = 2  # 2:1 ratio (fake:real) for reasonably balanced dataset

if os.path.exists(CELEBDF_PATH):
    print('\n=== Processing Celeb-DF ===')
    print(f"Configuration: {opts['max_videos_per_type'] or 'ALL'} real videos, "
          f"{opts['max_fake_videos_per_real']} fake video(s) per real, "
          f"{opts['frames_per_video']} frames per video")
    print(f"Expected output: ~{590 * opts['frames_per_video']} real frames + "
          f"~{590 * opts['max_fake_videos_per_real'] * opts['frames_per_video']} fake frames")
    
    # The process_celebdf_to_images function will save its own CSV `labels_celebdf.csv`
    df_meta_celeb = process_celebdf_to_images(
        CELEBDF_PATH, 
        IMAGES_DIR, 
        METADATA_DIR,
        max_videos_per_subject=opts['max_videos_per_type'],
        frames_per_video=opts['frames_per_video'],
        img_size=opts['img_size'],
        face_margin=opts['face_margin'],
        min_confidence=opts['min_confidence'],
        resize_max_width=opts['resize_max_width'],
        save_format=opts['save_format'],
        overwrite=opts['overwrite'],
        max_fake_videos_per_real=opts['max_fake_videos_per_real'],
        jpeg_quality=opts.get('jpeg_quality', 95)
    )
else:
    print('CELEBDF_PATH not found; skipping Celeb-DF processing:', CELEBDF_PATH)

# Show short previews of generated metadata
if 'df_meta_celeb' in locals() and df_meta_celeb is not None and not df_meta_celeb.empty:
    print('\n=== Celeb-DF Extraction Summary ===')
    print(f"Total frames extracted: {len(df_meta_celeb)}")
    print(f"Real frames (label=0): {len(df_meta_celeb[df_meta_celeb['label']==0])}")
    print(f"Fake frames (label=1): {len(df_meta_celeb[df_meta_celeb['label']==1])}")
    print(f"\nDataFrame preview:")
    display(df_meta_celeb.head(10))
    print(f"\nLabel distribution:\n{df_meta_celeb['label'].value_counts()}")
    print(f'\nMetadata saved to:')
    print(f"  - {os.path.join(METADATA_DIR, 'labels_celebdf.csv')}")
    print(f"  - {os.path.join(METADATA_DIR, 'detections_celebdf.json')}")
else:
    print('No Celeb-DF crops created or processing skipped.')


=== Processing Celeb-DF ===
Configuration: ALL real videos, 2 fake video(s) per real, 10 frames per video
Expected output: ~5900 real frames + ~11800 fake frames
Initializing MTCNN detector...


I0000 00:00:1764499876.684288      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1764499876.684987      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Face detector initialized.
Processing 590 Celeb-DF real videos from /kaggle/input/celeb-df-v2/Celeb-real


celeb_map_synth: 100%|██████████| 590/590 [43:58<00:00,  4.47s/it]


Saved 17481 rows to /kaggle/working/output/metadata/labels_celebdf.csv

=== Celeb-DF Extraction Summary ===
Total frames extracted: 17481
Real frames (label=0): 5883
Fake frames (label=1): 11598

DataFrame preview:


,filepath,video_id,frame_idx,bbox_x,bbox_y,bbox_w,bbox_h,confidence,timestamp,label,method
0,output/images/celebdf/real/id49_0006_0000.jpg,id49_0006,0,370,31,112,144,0.999728,0.000000,0,celeb_real
1,output/images/celebdf/real/id49_0006_0001.jpg,id49_0006,36,339,25,113,146,0.999995,1.200000,0,celeb_real
2,output/images/celebdf/real/id49_0006_0002.jpg,id49_0006,73,364,25,129,164,0.999972,2.433333,0,celeb_real
3,output/images/celebdf/real/id49_0006_0003.jpg,id49_0006,109,341,37,115,148,0.999906,3.633333,0,celeb_real
4,output/images/celebdf/real/id49_0006_0004.jpg,id49_0006,146,355,27,112,146,0.999901,4.866667,0,celeb_real
5,output/images/celebdf/real/id49_0006_0005.jpg,id49_0006,182,361,29,115,152,0.991117,6.066667,0,celeb_real
6,output/images/celebdf/real/id49_0006_0006.jpg,id49_0006,219,355,33,112,149,0.998876,7.300000,0,celeb_real
7,output/images/celebdf/real/id49_0006_0007.jpg,id49_0006,255,374,40,114,153,0.995707,8.500000,0,celeb_real
8,output/images/celebdf/real/id49_0006_0008.jpg,id49_0006,292,364,39,116,150,0.999817,9.733333,0,celeb_real
9,output/images/celebdf/real/id49_0006_0009.jpg,id49_0006,329,359,32,111,148,0.997805,10.966667,0,celeb_real



Label distribution:
label
1    11598
0     5883
Name: count, dtype: int64

Metadata saved to:
  - /kaggle/working/output/metadata/labels_celebdf.csv
  - /kaggle/working/output/metadata/detections_celebdf.json
